<a href="https://colab.research.google.com/github/xprilion/flower-species-recognition/blob/master/PlantSpeciesRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tf-nightly-gpu-2.0-preview
import tensorflow as tf
import os

In [0]:
_URL = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

zip_file = tf.keras.utils.get_file(origin=_URL, 
                                   fname="flower_photos.tgz", 
                                   extract=True, cache_subdir='/content',)

base_dir = os.path.join(os.path.dirname(zip_file), 'flower_photos')

In [3]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='training')

val_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='validation')

Found 2939 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


In [4]:
for image_batch, label_batch in train_generator:
  break
image_batch.shape, label_batch.shape

((64, 224, 224, 3), (64, 5))

In [5]:
print (train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(labels)

{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}


In [0]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                              include_top=False, 
                                              weights='imagenet')

In [0]:
base_model.trainable = False

In [0]:
model = tf.keras.Sequential([
      base_model,
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(5, activation='softmax')
])

In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [0]:
epochs = 10

history = model.fit(train_generator, 
                    epochs=epochs, 
                    validation_data=val_generator)

Epoch 1/10


W0527 18:54:59.375558 140092283127680 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


46/46 [==============================] - 417s 9s/step - loss: 1.1518 - accuracy: 0.6026 - val_loss: 1.2285 - val_accuracy: 0.5472
Epoch 2/10
46/46 [==============================] - 409s 9s/step - loss: 0.4944 - accuracy: 0.8237 - val_loss: 0.7724 - val_accuracy: 0.7278
Epoch 3/10
46/46 [==============================] - 411s 9s/step - loss: 0.3439 - accuracy: 0.8840 - val_loss: 0.7037 - val_accuracy: 0.7592
Epoch 4/10
46/46 [==============================] - 402s 9s/step - loss: 0.2219 - accuracy: 0.9224 - val_loss: 0.7978 - val_accuracy: 0.7209
Epoch 5/10
46/46 [==============================] - 402s 9s/step - loss: 0.2032 - accuracy: 0.9296 - val_loss: 1.0823 - val_accuracy: 0.7004
Epoch 6/10
46/46 [==============================] - 403s 9s/step - loss: 0.2134 - accuracy: 0.9282 - val_loss: 1.0036 - val_accuracy: 0.7100
Epoch 7/10
46/46 [==============================] - 401s 9s/step - loss: 0.1640 - accuracy: 0.9466 - val_loss: 0.6900 - val_accuracy: 0.7756
Epoch 8/10
46/46 [======

In [0]:
saved_model_dir = ''
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [0]:
from google.colab import files

files.download('model.tflite')
files.download('labels.txt')